In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import statistics

In [20]:
# Given array of equities, returning dataframe with historical data dating back 
def get_closing_price_historical_data(equities, period, interval):
    data = {}
    for x in equities:
        ticker = yf.Ticker(x)
        history = ticker.history(period= period, interval= interval)['Close']
        data[x] = history
    return data

# Given a m x n matrix  of m closing prices for n equities, returns an 1 x n 
# array of average closing prices
def compute_average(closing_prices_for_multiple_equities):
    # print(closing_prices_for_multiple_equities)
    sums = [0] * len(closing_prices_for_multiple_equities[0])
    for row in closing_prices_for_multiple_equities:
        for i in range(len(row)):
            sums[i] += row[i]

    avgs = [x/len(closing_prices_for_multiple_equities) for x in sums]
    return avgs

# Given an m x n matrix of m closing prices for n equities and n averages for n 
# equities, returns all of the difference of the closing prices and their averages
# essentially substracts mean to make the new mean of the data set 0 for each column
def compute_stocks_demeaned(closing_prices_matrix, average_closing_prices):
    m = len(closing_prices_matrix)
    n = len(closing_prices_matrix[0])
    demeaned = [[0 for i in range(n)] for j in range(m)]
    for i in range(m):
        for j in range(n):
            demeaned[i][j] = closing_prices_matrix[i][j] - average_closing_prices[j]

    return demeaned

# Given an m x n matrix of the demeaned closing prices for n equities, returns
# the associated covariance matrix calculated by demeaned transpose x demeaned.
def compute_covariance_matrix(demeaned):
    s_minus_m = np.array(demeaned)
    s_minus_m_t = s_minus_m.transpose()
    m = len(demeaned)
    return np.matmul(s_minus_m_t, s_minus_m)/m

def expected_portfolio_risk_based_on_stddev (asset_weights, covariance_matrix):
    Wt = np.asmatrix(asset_weights)
    W = Wt.transpose()
    return np.sqrt(Wt.dot(covariance_matrix).dot(W))

def expected_portfolio_return(average_closing_prices, asset_weights):
    M = np.array(average_closing_prices)
    W = np.array(asset_weights)
    return M.dot(W)

# convert from closing prices to percent change day over day
def convert_closing_prices_to_pct_change(closing_prices_matrix):
    for sym in closing_prices_matrix.columns:
        data = list(portfolio[sym])
        pct_change = []
        for i in range(1, len(data)):
            curr = data[i]
            yester = data[i-1]
            delta = curr-yester
            pct_change.append(delta/curr * 100)
        pct_change.insert(0, 0)
        closing_prices_matrix[sym] = pct_change
    return closing_prices_matrix

def sharpe_ratio(closing_prices_matrix, asset_weights, risk_free_rate):
    closing_prices_matrix = convert_closing_prices_to_pct_change(closing_prices_matrix)
    average_closing_prices = compute_average(closing_prices_matrix)
    demeaned = compute_stocks_demeaned(closing_prices_matrix, average_closing_prices)
    cov = compute_covariance_matrix(demeaned)
    risk = expected_portfolio_risk_based_on_stddev(asset_weights, cov)
    ret = expected_portfolio_return(average_closing_prices, asset_weights)
    # print(risk, ret, risk_free_rate)
    return (ret - risk_free_rate)/risk

In [21]:
trimmed_senbet = pd.read_excel('Senbet Portfolio.xlsx')
trimmed_senbet.columns = ['Ticker', 'Allocation', 'Shares']
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.O', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.K', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace(' ', ''), axis = 1)
asset_weights = list(trimmed_senbet['Allocation'])

In [22]:
historical_data = get_closing_price_historical_data(trimmed_senbet['Ticker'], '200d', '1d')

In [28]:
portfolio = pd.DataFrame(historical_data)

In [25]:
# sharpe_ratio(portfolio, asset_weights, 0.014)

KeyboardInterrupt: 

In [29]:
closing_prices_matrix = convert_closing_prices_to_pct_change(portfolio)
average_closing_prices = compute_average(closing_prices_matrix)
print(average_closing_prices)

KeyError: 0